In [4]:
import os 
from dotenv import load_dotenv
load_dotenv(override=True)
import requests
import json


In [5]:
OPEN_WEATHER_KEY = os.getenv("OPEN_WEATHER_KEY")
print(OPEN_WEATHER_KEY)


660f775a7383aad35872620768d2aa33


In [6]:
# Step 1.构建请求
url = "https://api.openweathermap.org/data/2.5/weather"

# Step 2.设置查询参数
params = {
"lat":"32.06",   # 查询南京实时天气
"lon":"118.80",
"appid": OPEN_WEATHER_KEY,# 输入API key
"units": "metric",# 使用摄氏度而不是华氏度
"lang": "zh_cn"# 输出语言为简体中文
}

# Step 3.发送GET请求
response = requests.get(url, params=params)

# Step 4.解析响应
data = response.json()
print(data)

{'coord': {'lon': 118.8, 'lat': 32.06}, 'weather': [{'id': 800, 'main': 'Clear', 'description': '晴', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 35.85, 'feels_like': 42.85, 'temp_min': 35.85, 'temp_max': 35.85, 'pressure': 1008, 'humidity': 52, 'sea_level': 1008, 'grnd_level': 1006}, 'visibility': 10000, 'wind': {'speed': 3.23, 'deg': 213, 'gust': 4.2}, 'clouds': {'all': 1}, 'dt': 1755575127, 'sys': {'country': 'CN', 'sunrise': 1755552708, 'sunset': 1755600330}, 'timezone': 28800, 'id': 1799962, 'name': 'Nanjing', 'cod': 200}


LangChain 调用外部工具流程

In [7]:
import os
from dotenv import load_dotenv 
load_dotenv(override=True)

OPEN_WEATHER_KEY = os.getenv("OPEN_WEATHER_KEY")
print(OPEN_WEATHER_KEY)


660f775a7383aad35872620768d2aa33


In [8]:
import os
import requests
import json
from langchain_core.tools import tool

@tool
def get_weather(lat, lon):

  """
  查询即时天气函数
  :param lat : 必要参数，字符串类型，用于表示查询天气的具体纬度，\
  :param lon : 必要参数，字符串类型，用于表示查询天气的具体经度，\

  :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
  返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
  """
  # Step 1.构建请求
  url = "https://api.openweathermap.org/data/2.5/weather"

  # Step 2.设置查询参数
  params = {
  "lat": lat,   # 纬度
  "lon": lon,   # 经度
  "appid": OPEN_WEATHER_KEY,# 输入API key
  "units": "metric",# 使用摄氏度而不是华氏度
  "lang":"zh_cn"# 输出语言为简体中文
  }

  # Step 3.发送GET请求
  response = requests.get(url, params=params)

  # Step 4.解析响应
  data = response.json()
  return json.dumps(data)

In [9]:
from langchain.chat_models import init_chat_model

# 初始化模型
model = init_chat_model("deepseek-chat", model_provider="deepseek")

In [10]:
# 定义 天气查询 工具函数
tools = [get_weather]

# 将工具绑定到模型
llm_with_tools = model.bind_tools(tools)

In [11]:
response = llm_with_tools.invoke("请问南京的天气如何?")
print(response)
print(response.additional_kwargs)


content='' additional_kwargs={'tool_calls': [{'id': 'call_0_34a40ef0-4bd3-4293-8315-82c8556e58f4', 'function': {'arguments': '{"lat":"32.0603","lon":"118.7969"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 202, 'total_tokens': 231, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 64}, 'prompt_cache_hit_tokens': 64, 'prompt_cache_miss_tokens': 138}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': 'da95ca5e-b572-4cfd-a61b-611e927f661c', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run--53cffc49-a477-412e-8008-0ea47cdf7df2-0' tool_calls=[{'name': 'get_weather', 'args': {'lat': '32.0603', 'lon': '118.7969'}, 'id': 'call_0_34a40ef0-4bd3-4293-8315-82c8556e58f4', 'type': 'tool_call'}] usage_metadata={'input_tokens': 202, 'output_tokens': 29, 'total_tokens': 

  我们需要调用ToolsAgentOutputParser输出解析器来处理模型响应。

In [12]:
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser

# 解析模型响应
agentAction = ToolsAgentOutputParser().invoke(response)

print(agentAction)

[ToolAgentAction(tool='get_weather', tool_input={'lat': '32.0603', 'lon': '118.7969'}, log="\nInvoking: `get_weather` with `{'lat': '32.0603', 'lon': '118.7969'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_34a40ef0-4bd3-4293-8315-82c8556e58f4', 'function': {'arguments': '{"lat":"32.0603","lon":"118.7969"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 202, 'total_tokens': 231, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 64}, 'prompt_cache_hit_tokens': 64, 'prompt_cache_miss_tokens': 138}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': 'da95ca5e-b572-4cfd-a61b-611e927f661c', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--53cffc49-a477-412e-8008-0ea47cdf7df2-0', tool_calls=[{'name': 'get_weather'

In [13]:
for tool_call in response.tool_calls:
  print(tool_call["args"])

{'lat': '32.0603', 'lon': '118.7969'}


In [14]:
# 获取工具调用
for tool_call in response.tool_calls:
  selected_tool = {"get_weather": get_weather}["get_weather"]

tool_output = selected_tool.invoke(tool_call["args"])
print(tool_output)

{"coord": {"lon": 118.8, "lat": 32.06}, "weather": [{"id": 800, "main": "Clear", "description": "\u6674", "icon": "01d"}], "base": "stations", "main": {"temp": 35.85, "feels_like": 42.85, "temp_min": 35.85, "temp_max": 35.85, "pressure": 1008, "humidity": 52, "sea_level": 1008, "grnd_level": 1006}, "visibility": 10000, "wind": {"speed": 3.23, "deg": 213, "gust": 4.2}, "clouds": {"all": 1}, "dt": 1755575127, "sys": {"country": "CN", "sunrise": 1755552708, "sunset": 1755600330}, "timezone": 28800, "id": 1799962, "name": "Nanjing", "cod": 200}


agent_action：AgentAction 对象
‑ tool: 要调用的工具名
‑ tool_input: dict，传给工具的参数
‑ log: 字符串，agent 在思考时写的 log（供人看）
tool_output：工具返回的字符串 / dict / 任意对象

In [15]:
from langchain.agents.format_scratchpad.tools import format_to_tool_messages

format_to_tool_messages(intermediate_steps = [(agentAction[0], tool_output)])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_34a40ef0-4bd3-4293-8315-82c8556e58f4', 'function': {'arguments': '{"lat":"32.0603","lon":"118.7969"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 202, 'total_tokens': 231, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 64}, 'prompt_cache_hit_tokens': 64, 'prompt_cache_miss_tokens': 138}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': 'da95ca5e-b572-4cfd-a61b-611e927f661c', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--53cffc49-a477-412e-8008-0ea47cdf7df2-0', tool_calls=[{'name': 'get_weather', 'args': {'lat': '32.0603', 'lon': '118.7969'}, 'id': 'call_0_34a40ef0-4bd3-4293-8315-82c8556e58f4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 202, 'output_tokens': 29, 

AIMessage 代表“大模型决定调用工具”
ToolMessage 代表“工具实际返回的结果”   
agent_scratchpad Agent 的“草稿纸”——一个临时变量，用来把已经发生的工具调用及其结果拼成一段文本或消息列表，再塞回给 LLM，让它在下一轮还能“看到”之前干了什么，从而继续推理。

历次 AgentAction（工具名、参数、LLM 当时的思考 log）
对应的 Observation（工具返回的内容）
按 ReAct 格式或 OpenAI 函数调用格式组织好，作为上下文追加到提示里
用户问问题
   ↓
AgentExecutor 组装 prompt → 把 agent_scratchpad 填进去
   ↓
LLM 决定调用工具
   ↓
工具执行，得到 observation
   ↓
新的 (AgentAction, observation) 被追加到 intermediate_steps
   ↓
再次被 format 成 agent_scratchpad
   ↓
循环，直到 LLM 给出 Final Answer

In [16]:
from langchain.agents import create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate

#定义工具
tools = [get_weather]

# 构建提示模版
prompt = ChatPromptTemplate.from_messages([
("system", "你是天气助手，请根据用户的问题，给出相应的天气信息"),
("human", "{input}"),
("placeholder", "{agent_scratchpad}"),
])

# 初始化模型
model = init_chat_model("deepseek-chat", model_provider="deepseek")

# 直接使用`create_tool_calling_agent`创建代理
agent = create_tool_calling_agent(model, tools, prompt)

  使用AgentExecutor来执行代理。代码如下：

In [17]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


response = agent_executor.invoke({"input": "请问今天南京的天气怎么样？"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'lat': '32.0603', 'lon': '118.7969'}`


{"coord": {"lon": 118.8, "lat": 32.06}, "weather": [{"id": 800, "main": "Clear", "description": "\u6674", "icon": "01d"}], "base": "stations", "main": {"temp": 35.85, "feels_like": 42.85, "temp_min": 35.85, "temp_max": 35.85, "pressure": 1008, "humidity": 52, "sea_level": 1008, "grnd_level": 1006}, "visibility": 10000, "wind": {"speed": 3.23, "deg": 213, "gust": 4.2}, "clouds": {"all": 1}, "dt": 1755575127, "sys": {"country": "CN", "sunrise": 1755552708, "sunset": 1755600330}, "timezone": 28800, "id": 1799962, "name": "Nanjing", "cod": 200}今天南京的天气是晴天，气温为35.85°C，体感温度较高，达到42.85°C。湿度为52%，风速为3.23米/秒，风向为213度。能见度为10000米，气压为1008 hPa。整体天气状况良好，适合出行，但请注意防晒和补水。

> Finished chain.
{'input': '请问今天南京的天气怎么样？', 'output': '今天南京的天气是晴天，气温为35.85°C，体感温度较高，达到42.85°C。湿度为52%，风速为3.23米/秒，风向为213度。能见度为10000米，气压为1008 hPa。整体天气状况良好，适合出行，但请注意防晒和补水。'}


In [18]:
print(response["output"])

今天南京的天气是晴天，气温为35.85°C，体感温度较高，达到42.85°C。湿度为52%，风速为3.23米/秒，风向为213度。能见度为10000米，气压为1008 hPa。整体天气状况良好，适合出行，但请注意防晒和补水。


![agentexecutor.png](image/README/AgentExecutor.png)

  在Agents的内部结构。每个Agent组件一般会由语言模型 + 提示 + 输出解析器构成，它会作为Agents的大脑去处理用户的输入。Agent能够处理的输入主要来源于三个方面：input代表用户的原始输入，Model Response指的是模型对某一个子任务的响应输出，而History则能携带上下文的信息。其输出部分，则链接到实际的工具库，需要调用哪些工具，将由经过Agent模块后拆分的子任务来决定。
  而我们知道，大模型调用外部函数会分为两个过程：识别工具和实际执行。在Message -> Agent -> Toolkits 这个流程中，负责的是将子任务拆解，然后根据这些子任务在工具库中找到相应的工具，提取工具名称及所需参数，这个过程可以视作一种“静态”的执行流程。而将这些决策转化为实际行动的工作，则会交给AgentExecutor。
  所以综上需要理解的是：在LangChain的Agents实际架构中，Agent的角色是接收输入并决定采取的操作，但它本身并不直接执行这些操作。这一任务是由AgentExecutor来完成的。将Agent（决策大脑）与AgentExecutor（执行操作的Runtime）结合使用，才构成了完整的Agents（智能体），其中AgentExecutor负责调用代理并执行指定的工具，以此来实现整个智能体的功能

In [ ]:
from langchain.agents import create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate

#定义工具
tools = [get_weather]

# 构建提示模版
prompt = ChatPromptTemplate.from_messages([
("system", "你是天气助手，请根据用户的问题，给出相应的天气信息"),
("human", "{input}"),
("placeholder", "{agent_scratchpad}"),
])

# 初始化模型
model = init_chat_model("deepseek-chat", model_provider="deepseek")

# 直接使用`create_tool_calling_agent`创建代理
agent = create_tool_calling_agent(model, tools, prompt)

create_tool_calling_agent中，已经自动处理了{并行工具}调用的处理逻辑，并不需要我们在手动处理，比如接下来测试一些复杂的问题

In [20]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


response = agent_executor.invoke({"input": "请问今天南京和郑州的天气怎么样，哪个城市更热？"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'lat': '32.0603', 'lon': '118.7969'}`


{"coord": {"lon": 118.8, "lat": 32.06}, "weather": [{"id": 800, "main": "Clear", "description": "\u6674", "icon": "01d"}], "base": "stations", "main": {"temp": 35.85, "feels_like": 42.85, "temp_min": 35.85, "temp_max": 35.85, "pressure": 1008, "humidity": 52, "sea_level": 1008, "grnd_level": 1006}, "visibility": 10000, "wind": {"speed": 3.23, "deg": 213, "gust": 4.2}, "clouds": {"all": 1}, "dt": 1755575127, "sys": {"country": "CN", "sunrise": 1755552708, "sunset": 1755600330}, "timezone": 28800, "id": 1799962, "name": "Nanjing", "cod": 200}
Invoking: `get_weather` with `{'lat': '34.7466', 'lon': '113.6253'}`


{"coord": {"lon": 113.6253, "lat": 34.7466}, "weather": [{"id": 802, "main": "Clouds", "description": "\u591a\u4e91", "icon": "03d"}], "base": "stations", "main": {"temp": 36.33, "feels_like": 42.63, "temp_min": 36.33, "temp_max": 36.33, "pressure": 1005, "humidity": 47

In [21]:
print(response["output"])

今天南京和郑州的天气情况如下：

- **南京**：
  - 天气：晴
  - 温度：35.85°C（体感温度42.85°C）
  - 湿度：52%
  - 风速：3.23 m/s

- **郑州**：
  - 天气：多云
  - 温度：36.33°C（体感温度42.63°C）
  - 湿度：47%
  - 风速：7.07 m/s

**对比**：郑州的温度略高于南京（36.33°C vs 35.85°C），因此郑州更热一些。


![binglian.png](image/README/并联调用.png)

In [22]:
@tool
def write_file(content):
    """
    将指定内容写入本地文件。
    :param content: 必要参数，字符串类型，用于表示需要写入文档的具体内容。
    :return：是否成功写入
    """

    return "已成功写入本地文件。"

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate


tools = [get_weather, write_file]

prompt = ChatPromptTemplate.from_messages([
("system", "你是天气助手，请根据用户的问题，给出相应的天气信息，如果用户需要将查询结果写入文件，请使用write_file工具"),
("human", "{input}"),
("placeholder", "{agent_scratchpad}"),
])

# 初始化模型
model = init_chat_model("deepseek-chat", model_provider="deepseek")

agent = create_tool_calling_agent(model, tools, prompt)

In [24]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": "查一下南京和北京的天气如何,记录在本地文件中?"})

print(response)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'lat': '32.0603', 'lon': '118.7969'}`


{"coord": {"lon": 118.8, "lat": 32.06}, "weather": [{"id": 800, "main": "Clear", "description": "\u6674", "icon": "01d"}], "base": "stations", "main": {"temp": 35.85, "feels_like": 42.85, "temp_min": 35.85, "temp_max": 35.85, "pressure": 1008, "humidity": 52, "sea_level": 1008, "grnd_level": 1006}, "visibility": 10000, "wind": {"speed": 3.23, "deg": 213, "gust": 4.2}, "clouds": {"all": 1}, "dt": 1755575127, "sys": {"country": "CN", "sunrise": 1755552708, "sunset": 1755600330}, "timezone": 28800, "id": 1799962, "name": "Nanjing", "cod": 200}
Invoking: `get_weather` with `{'lat': '39.9042', 'lon': '116.4074'}`


{"coord": {"lon": 116.4074, "lat": 39.9042}, "weather": [{"id": 501, "main": "Rain", "description": "\u4e2d\u96e8", "icon": "10d"}], "base": "stations", "main": {"temp": 25.17, "feels_like": 26.14, "temp_min": 25.17, "temp_max": 25.17, "pressure": 1006, "humidity": 92, 